In [ ]:
import os
import time
import json
import asyncio
import datetime
from vertexai.generative_models import GenerativeModel, Part
import sys
sys.path.append('/home/idies/workspace/Storage/xyu1/persistent/GenCELLAgent/')

from src.tools.serp import search as google_search_summary
from src.tools.segment import segment_image 
from src.tools.segmentation_eval import segmentation_evaluator
from src.tools.oneshot_segGPT import seggpt_inference_img
from src.tools.mitonet import mitonet_inference
from src.tools.summarizer import summarizer_report
from src.tools.launch_human_correction import launch_sam_correction_tool


import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def compute_iou(pred_mask, gt_mask):
    pred = np.array(pred_mask) > 0
    gt = np.array(gt_mask) > 0

    intersection = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum()

    if union == 0:
        return 1.0 if intersection == 0 else 0.0  # handle empty masks
    return intersection / union


def get_score(outputs_second):
    evaluation_data = outputs_second.get("evaluation", {})
    score_second = None

    # Case 1: If evaluation_data is a dict with raw text fallback
    if isinstance(evaluation_data, dict):
        raw_text_second = evaluation_data.get("raw_text", "")
        cleaned_json_second = raw_text_second.strip('`').strip()

        # Remove "json" prefix if present
        if cleaned_json_second.startswith("json"):
            cleaned_json_second = cleaned_json_second[len("json"):].strip()

    # Case 2: If evaluation_data is just a string (raw JSON)
    elif isinstance(evaluation_data, str):
        cleaned_json_second = json.dumps(evaluation_data, indent=2)
    else:
        cleaned_json_second = ""

    # Try to find OverallScore in the cleaned text
    match_second = re.search(r'"OverallScore"\s*:\s*(\d+\.?\d*)', cleaned_json_second)
    if match_second:
        score_str_second = match_second.group(1)
        score_second = float(score_str_second)
    else:
        print("OverallScore not found in evaluation data.")
    return score_second

In [ ]:
input_filename = "golgi_apparatus_visual_characteristics.json"

# Read the file
with open(input_filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Now you can use `data` like a regular Python dictionary
print(data)

In [ ]:
result_dict = json.loads(data)

# Now extract the segmentation prompt
segmentation_prompt = result_dict["summary"]["segmentation_prompt"]
visual_characteristics = result_dict["summary"]["visual_characteristics"]

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
import os
from datetime import datetime
#[15,25,33,37,53,65,82,89,9,97]

for index in [25]:
    for test in range(1):
        # Create a timestamped directory for the current run
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_dir = f"./results_golgi/best_per_step_Gemini_2_5_init_new_ICT_{index}/run_{timestamp}"
        os.makedirs(save_dir, exist_ok=True)

        # Setup
        base_img_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/images/image_{}.png"
        base_label_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/labels/label_{}.png"
        segment_save_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/segment_resutls/results.png"
        segment_mask_path = "/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/segment_resutls/results_mask.png"

        image_example_1_path = f"/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/images/image_{index}.png"
        image_example_2_path = f"/home/idies/workspace/Storage/xyu1/persistent/Langchain/ours_test/test_images/golgi/overlay_all/image_{index}.png"
        evaluation_prompt_path = "in_context_prompt_golgi.json"

        init_seg_path = os.path.join(save_dir, f"initial_segment_step_0_iter_0.png")
        init_mask_path = os.path.join(save_dir, f"initial_mask_step_0_iter_0.png")
        init_gt_path = os.path.join(save_dir, f"ground_truth_step_0_iter_0.png")
        init_prompt_path = os.path.join(save_dir, f"initial_prompt_step_0_iter_0.txt")
        init_IOU_path = os.path.join(save_dir, f"IOU_step_0_iter_0.txt")


        image_path = base_img_path.format(index)
        ground_truth_path = base_label_path.format(index)

        n_steps = 5
        n_trials_per_step = 5

        # Replace these with actual objects
        current_prompt = segmentation_prompt  # make sure this is defined

        all_step_results = []
        prompt_history = []

        for step in range(n_steps):
            print(f"\n========== Step {step + 1} ==========")

            step_best_score = float('-inf')
            step_best_mask = None
            step_best_segment = None
            step_best_prompt = current_prompt
            step_best_ground_truth = np.array(Image.open(ground_truth_path))


            #save_all step images
            step_dir = os.path.join(save_dir, f"step_{step+1}")
            os.makedirs(step_dir, exist_ok=True)


            for iteration in range(n_trials_per_step):
                print(f"\n--- Iteration {iteration + 1} ---")
                iter_dir = os.path.join(step_dir, f"iter_{iteration+1}")
                os.makedirs(iter_dir, exist_ok=True)
                
                #1 first segment the init the image with tht init prompt
                segment_image(current_prompt, image_path)

                if step == 0 and iteration == 0:
                    # Load images
                    segmentation_image = Image.open(segment_save_path)
                    mask_image = Image.open(segment_mask_path)
                    ground_truth_label = np.array(Image.open(ground_truth_path))

                    iou_score = compute_iou(mask_image, ground_truth_label)
                    print(f"📊 init IoU: {iou_score:.4f}")

                    # Save IoU to file
                    with open(init_IOU_path, "w") as f:
                        f.write(f"IoU: {iou_score:.4f}")

                    #segmentation_image.save(init_seg_path)
                    mask_image.save(init_mask_path)
                    Image.fromarray(ground_truth_label).save(init_gt_path)
                    segmentation_image.save(init_seg_path)

                    with open(init_prompt_path, "w") as pf:
                        pf.write(current_prompt)

                #1 first evalautor the init image segmentation to generate the refine prompt
                outputs_init = segmentation_evaluator(visual_characteristics, 
                                                 segment_save_path, 
                                                 current_prompt,
                                                 evaluation_prompt_path,
                                                 image_example_1_path,
                                                 image_example_2_path)

                refined_prompt = outputs_init["refined_segmentation_prompt"]
                
                if step == 0:
                    raw_text = outputs_init["evaluation"].get("raw_text", "")
                    cleaned_json = raw_text.strip('`').strip()

                    # Step 3: Remove optional "json" prefix if present
                    if cleaned_json.startswith("json"):
                        cleaned_json = cleaned_json[len("json"):].strip()

                    match = re.search(r'"OverallScore"\s*:\s*(\d+\.?\d*)', cleaned_json)
                    if match:
                        score_str = match.group(1) if match.group(1) is not None else match.group(2)
                        score = float(score_str)
                    
                    segmentation_image.save(os.path.join(iter_dir, f"step_0_iter_{iteration}_score_{score}.png"))

                
                

                #2 second segment image with refine prompt
                segment_image(refined_prompt, image_path)
                                            
                                            
                outputs_second = segmentation_evaluator(visual_characteristics, 
                                                 segment_save_path, 
                                                 refined_prompt,
                                                 evaluation_prompt_path,
                                                 image_example_1_path,
                                                 image_example_2_path)
                                            
                                            
                segmentation_image = Image.open(segment_save_path).copy()
                mask_image = Image.open(segment_mask_path).convert('L').copy()
                mask_array = np.array(mask_image, dtype=np.uint8)


                total_pixels = mask_array.size
                foreground_pixels = np.sum(mask_array > 0)
                foreground_ratio = foreground_pixels / total_pixels

                raw_text_second = outputs_second["evaluation"].get("raw_text", "")
                cleaned_json_second = raw_text_second.strip('`').strip()

                # Step 3: Remove optional "json" prefix if present
                if cleaned_json_second.startswith("json"):
                    cleaned_json_second = cleaned_json_second[len("json"):].strip()

                match_second = re.search(r'"OverallScore"\s*:\s*(\d+\.?\d*)', cleaned_json_second)
                if match_second:
                    score_str_second = match_second.group(1) if match_second.group(1) is not None else match_second.group(2)
                    score_second = float(score_str_second)

                print(f"Foreground ratio: {foreground_ratio:.4f}, Score: {score_second:.2f}")

                mask_path = os.path.join(iter_dir, f"mask_ratio_{foreground_ratio:.4f}_score_{score_second}.png")
                segment_path = os.path.join(iter_dir, f"segment_ratio_{foreground_ratio:.4f}_score_{score_second}.png")
                prompt_path = os.path.join(iter_dir, f"prompt_ratio_{foreground_ratio:.4f}_score_{score_second}.txt")
                review_path = os.path.join(iter_dir, f"review_path_ratio_{foreground_ratio:.4f}_score_{score_second}.txt")
                mask_image.save(mask_path)
                segmentation_image.save(segment_path)

                with open(prompt_path, "w") as pf:
                    pf.write(refined_prompt)

                with open(review_path, "w", encoding="utf-8") as pf:
                    pf.write(raw_text_second)



                if 0.15 <= foreground_ratio <= 0.75:
                    if score_second > step_best_score:
                        step_best_score = score_second
                        step_best_mask = mask_image.copy()
                        step_best_segment = segmentation_image.copy()
                        step_best_prompt = refined_prompt  # track best prompt for next step
                        print("✅ New best saved for this step.")

                    else:
                        print("ℹ️ Meets condition but not highest score.")
                else:
                    print("❌ Skipped due to poor foreground ratio.")



            # Use best prompt from this step for next step
            current_prompt = step_best_prompt
            prompt_history.append({
                "step": step + 1,
                "used_prompt": current_prompt
            })

            print(f"\n🔁 Final prompt used for Step {step + 1}:")
            print(current_prompt)

            if step_best_mask and step_best_segment:
                mask_path = os.path.join(save_dir, f"best_mask_step_{step+1}_iter{iteration+1}.png")
                segment_path = os.path.join(save_dir, f"best_segment_step_{step+1}_iter{iteration+1}.png")
                prompt_path = os.path.join(save_dir, f"best_prompt_step_{step+1}_iter{iteration+1}.txt")
                iou_path = os.path.join(save_dir, f"best_prompt_IOU_step_{step+1}_iter{iteration+1}.txt")
                step_best_mask.save(mask_path)
                step_best_segment.save(segment_path)

                with open(prompt_path, "w") as pf:
                    pf.write(step_best_prompt)


                iou_score = compute_iou(step_best_mask, step_best_ground_truth)

                # Save IoU to file
                with open(iou_path, "w") as f:
                    f.write(f"IoU: {iou_score:.4f}")

                all_step_results.append({
                    "step": step + 1,
                    "segmentation_image": step_best_segment,
                    "mask_image": step_best_mask,
                    "ground_truth": step_best_ground_truth
                })

        print("\n=== Final Best Refined Prompt ===")
        print(current_prompt)
